# **Automatic Safety Gear Detection in Industrial Settings**

In [ ]:
import cv2
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO('best.pt')

# Load background image
imgBg = cv2.imread("graphic/bg720.png")

# Define the class names and their corresponding colors
CLASS_NAMES = ['hard-hat', 'jacket', 'mask', 'no-hard-hat', 'no-jacket', 'no-mask', 'person']
CLASS_COLORS = {'hard-hat': (0, 200, 0), 'no-hard-hat': (0, 0, 200), 'jacket': (0, 200, 0), 'no-jacket': (0, 0, 200),
                'mask': (0, 200, 0), 'no-mask': (0, 0, 200), 'person': (250, 250,0)}

# Upload videos / images
#cap = cv2.VideoCapture("graphic/4.jpg")
#cap = cv2.VideoCapture("Videos/video2.mp4")

# Set up camera and resolution
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    try:
        success, img = cap.read()     # Read a frame from the video capture
        img = cv2.resize(img, (640, 480))  # Resize the image
        results = model(img, stream=True)    # Use the model to detect objects in the frame
        imgBg[162:162+480, 65:65+640] = img     # Overlay the detected objects on the background image
        
        # Verification    
        detected_person = False
        detected_mask = False
        detected_hard_hat = False
        detected_jacket = False
        
        for r in results:
            # Get the bounding box coordinates of the object
            boxes = r.boxes
                        
            for box in boxes:
                x1, y1, x2, y2 =box.xyxy[0]
                box_x1, box_y1, box_x2, box_y2 =  int(x1), int(y1),int(x2),int(y2)
                # Calculate the confidence of the object detection
                conf = int((box.conf[0]*100))/100
                # Get the class name of the object
                cls = int(box.cls[0])
                currentClass = CLASS_NAMES[cls]
                
                if conf > 0.5:
                    # Determine the color of the rectangle based on the class name
                    color = CLASS_COLORS[currentClass]
                    # Draw a rectangle around the detected object
                    bbox = (65 + box_x1, 162 + box_y1, box_x2 - box_x1, box_y2 - box_y1)
                    cv2.rectangle(imgBg, bbox, color, 3)
                    # Draw text on the rectangle with the class name and confidence
                    cv2.putText(imgBg, f'{currentClass} {conf:.2f}',(max(0, 66 + box_x1), max(35, 155 + box_y1)), cv2.FONT_HERSHEY_SIMPLEX,.6,color,1)   
                
                # Check if the detected object is a person, mask, hard-hat, or jacket
                if conf > 0.5:
                    if currentClass == 'person':
                        detected_person = True
                    elif currentClass == 'mask':
                        detected_mask = True
                    elif currentClass == 'hard-hat' :
                        detected_hard_hat = True
                    elif currentClass == 'jacket':
                        detected_jacket = True      
            
                        
            # Check for verification. Bottom part
            cv2.rectangle(imgBg,(190, 540), (580, 620), (0,0,0), -1) # Black Status Box
            if detected_person and detected_mask and detected_hard_hat and detected_jacket:
                cv2.putText(imgBg,'  Verified  ', (200, 600), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 200, 0), 5)
            else:
                cv2.putText(imgBg,'Not Verified', (200, 600), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 200), 5)   
                   
    except Exception as e:
        print(f"Error: {e}")

    # Display the output image
    cv2.imshow('Background', imgBg)
    # Export final images
    #cv2.imwrite('output4.jpg', imgBg)

    # Check if the user pressed the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera capture and close all OpenCV windowsq
cap.release()
cv2.destroyAllWindows()